## Author - Rhythm Girdhar

In [14]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from textblob import TextBlob


In [3]:
url = "https://www.aljazeera.com/where/mozambique/"
def getHTMLdocument(url):
    response = requests.get(url)
    return response.text

aljazeera_paper = getHTMLdocument(url)

In [4]:
base_url = "https://www.aljazeera.com"
soup = BeautifulSoup(aljazeera_paper, 'html.parser')
all_articles = soup.find_all("a", {"class": "u-clickable-card__link"})

article_links = [base_url + article['href'] for article in all_articles]
print(article_links)

['https://www.aljazeera.com/news/2022/5/23/floods-hit-south-africas-kwazulu-natal-province-again', 'https://www.aljazeera.com/news/2022/3/18/mozambique-cyclone-gombe-death-toll-rises-to-53', 'https://www.aljazeera.com/news/2022/3/4/mozambique-announces-new-prime-minister-and-finance-minister', 'https://www.aljazeera.com/economy/2022/3/1/analysis-can-african-gas-replace-russian-supplies-to-europe', 'https://www.aljazeera.com/news/2022/1/27/at-least-70-dead-from-tropical-storm-ana-in-southern-africa', 'https://www.aljazeera.com/news/2022/1/12/southern-africa-bloc-sadc-extends-mozambique-mission', 'https://www.aljazeera.com/program/start-here/2021/10/3/climate-change-and-famine-start-here', 'https://www.aljazeera.com/news/2021/9/24/in-mozambique-kagame-says-rwandan-troops-presence-to-continue', 'https://www.aljazeera.com/news/2021/8/8/rwanda-mozambique-forces-recapture-port-city-from-rebels', 'https://www.aljazeera.com/news/2021/7/10/rwanda-deploys-1000-soldiers-to-mozambique-cabo-delgado

In [69]:
from newspaper import Article
import json
from tqdm import tqdm

article_collections = []

# article_url = article_links[0]
# article = Article(article_url)
# article.download()
# article.parse()
# article_soup = BeautifulSoup(article.html, 'html.parser')
# article_soup.find("p", {"class": "article__subhead"}).find('em').string

for article_url in tqdm(article_links):
    article = Article(article_url)
    article.download()
    article.parse()
    article_soup = BeautifulSoup(article.html, 'html.parser')
    subtitle = ''
    try:
        subtitle =  article_soup.find("p", {"class": "article__subhead"}).find('em').string
    except:
        pass
    article_collections.append(
        { 'title': article.title, 'subtitle': subtitle, 'content': article.text }
    )

print(len(article_collections))
with open('data.json', 'w') as f:
    json.dump(article_collections, f)

# json.dumps(article_collections)

100%|██████████| 14/14 [00:03<00:00,  3.88it/s]

14


# Sentiment Analysis

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rhythmgirdhar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
import json
with open('data.json') as f:
    data = json.load(f)

# print(data)

In [9]:
stopwords = nltk.corpus.stopwords.words("english")

In [11]:
import copy

cleaned_data = copy.deepcopy(data)
preprocessed_data = []
for each in cleaned_data:
    each['content'] = nltk.word_tokenize(each['content'])
    each['content'] = [w for w in each['content'] if w.lower() not in stopwords]
    each['content'] = [w for w in each['content'] if w.lower() not in [",", ".", "(", ")", "”", "[", "]", "“", "’", ":", "–"]]
    preprocessed_data.append(" ".join([w.lower() for w in each['content']]))

# print(preprocessed_data)
print(len(preprocessed_data))

14


## Using NLTK Sentiment

### Sentiment Analysis using Article Content

In [66]:
from nltk.sentiment import f
from statistics import mean


nltk_sentiment = []
sia = SentimentIntensityAnalyzer()
for each in data:
    scores = [sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(each['content'])]
    print(mean(scores))
print("\n")
print("-------After Pre-processed Data-------")
print("\n")
for each in preprocessed_data:
    scores = [sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(each)]
    print(mean(scores))
    
    sentiment = mean(scores)
    temp = [sentiment]
    if sentiment < 0.5 and sentiment > -0.5:
        temp.append(0.0)
        temp.append("Neutral")
    elif sentiment > 0.0:
        temp.append(1.0)
        temp.append("Positive")
    elif sentiment < 0.0:
        temp.append(-1.0)
        temp.append("Negative")
    nltk_sentiment.append(temp)

-0.15685625
-0.7474714285714286
-0.03154000000000001
0.18046666666666666
-0.360745
0.160475
-0.31245
-0.16955135135135135
0.03320000000000001
-0.2301764705882353
-0.319995
-0.1600921875
-0.389468
-0.03635


-------After Pre-processed Data-------


-0.68235
-0.9933
-0.304
0.5121142857142857
-0.9957
0.946
-0.6249
-0.4469
0.5859
-0.7179
-0.9949
-0.418925
-0.9894499999999999
-0.3142333333333333


In [16]:
df = pd.DataFrame(nltk_sentiment, columns=['Value', 'Sentiment', 'Sentiment Catergory'])

In [17]:
import plotly.express as px

colors = ['gold', 'mediumturquoise', 'darkorange']
fig = px.pie(values=df['Sentiment'].value_counts().values, names=df['Sentiment Catergory'].unique(),
             color_discrete_sequence=px.colors.sequential.RdBu,
             title = 'Distribution Based on Sentiments')
fig.update_traces(textposition='inside', textinfo='percent+label', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

## Text Blob Sentiment

In [18]:
text_blob_sentiment = []
import pandas as pd
for each in data:
    sentiment = TextBlob(each['content']).sentiment.polarity
    print(sentiment)
    temp = []
    if sentiment < 0.05 and sentiment > -0.05:
        temp.append(0.0)
        temp.append("Neutral")
    elif sentiment > 0.0: 
        temp.append(1.0)
        temp.append("Positive")
    elif sentiment < 0.0:
        temp.append(-1.0)
        temp.append("Negative")
    text_blob_sentiment.append(temp)
    
df = pd.DataFrame(text_blob_sentiment, columns=['Value', 'Sentiment'])

0.06215277777777778
-0.1579545454545455
0.09696969696969698
0.06860242931671502
0.018749999999999992
0.10260416666666665
-0.5
0.03796825396825398
0.10328947368421053
-0.006414141414141413
-0.011750000000000012
0.05235521235521236
0.05724160322436185
-0.054368279569892486


In [19]:
import plotly.express as px


colors = ['gold', 'mediumturquoise', 'darkorange']
fig = px.pie(values=df['Value'].value_counts().values, names=df['Sentiment'].unique(),
             color_discrete_sequence=px.colors.sequential.RdBu,
             title = 'Distribution Based on Sentiments')
fig.update_traces(textposition='inside', textinfo='percent+label', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

## Sentiment Analysis using Title, Subtitle, and Content

In [27]:
content_score = []
for each in preprocessed_data:
    content_score.append([sia.polarity_scores(sentence)["compound"] for sentence in nltk.sent_tokenize(each)])

for each in data:  
    i = 0 
    title_score = sia.polarity_scores(each['title'])['compound']
    subtitle_score = sia.polarity_scores(each['subtitle'])['compound']
    total_score = (50 * mean(content_score[i])) + (25 * title_score) + (25 * subtitle_score)
    print(title_score, subtitle_score, content_score[i], total_score)
    i = i + 1

0.0 -0.6705 [-0.9628, -0.4019] -50.879999999999995
-0.5994 -0.8176 [-0.9628, -0.4019] -69.5425
0.0 0.0 [-0.9628, -0.4019] -34.1175
0.0 0.122 [-0.9628, -0.4019] -31.0675
-0.6486 -0.8555 [-0.9628, -0.4019] -71.72
0.128 0.25 [-0.9628, -0.4019] -24.6675
0.0 0.0 [-0.9628, -0.4019] -34.1175
0.0 0.2263 [-0.9628, -0.4019] -28.46
-0.2023 0.1779 [-0.9628, -0.4019] -34.7275
0.0 0.296 [-0.9628, -0.4019] -26.7175
0.3612 -0.8481 [-0.9628, -0.4019] -46.29
0.0 -0.7096 [-0.9628, -0.4019] -51.8575
-0.5423 0.25 [-0.9628, -0.4019] -41.425
0.4215 -0.0772 [-0.9628, -0.4019] -25.509999999999998
